In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 6.9 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=bf73fbb2298e2379246f69f0b398da9889cbe926b6699ea91251dbaa2867e021
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
requests 2.23.0 requires urllib3!=

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [2]:
!signate list
!signate files --competition-id=133
!signate download --competition-id=133

  competitionId  title                                                                             closing     prize        submitters
---------------  --------------------------------------------------------------------------------  ----------  ---------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                              -                              5893
             24  【練習問題】お弁当の需要予測                                                      -                              7340
             27  【練習問題】Jリーグの観客動員数予測                                               -                              1763
            100  【練習問題】手書き文字認識                                                        -           Knowledge           226
            102  【練習問題】タイタニックの生存予測                                                -           Knowledge          1686
            103  【練習問題】音楽ラベリング                                                        -           Knowledge            98
            

In [3]:
!unzip -q test_images.zip

In [4]:
# ディレクトリ内のデータの確認
%ls

label_master.tsv  sample_submit.tsv  test_images.zip   train_master.tsv
sample_data/      test_images/       train_images.zip


In [9]:
!pip install -q timm

     |████████████████████████████████| 509 kB 4.2 MB/s 
     |████████████████████████████████| 127 kB 70.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
signate 0.9.9 requires urllib3>=1.26.7, but you have urllib3 1.25.11 which is incompatible.


In [11]:
# ライブラリのインポート
import os
import random
from glob import glob
from warnings import filterwarnings

import argparse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import cv2
from PIL import Image

import timm

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchsummary import summary

from collections import defaultdict

filterwarnings('ignore')  # warningをオフにする

# ランダムシードの設定
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [6]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# モデルのパス
model_path = '/content/drive/MyDrive/Colab Notebooks/cifar10/model/convnext_base_loss_0.3857.pth'

In [7]:
# デバイスの設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} # GPUが使える場合、DataLoaderでのミニバッチの取り出しを高速化
print(f'デバイス：{device}')

デバイス：cuda


In [13]:
model = timm.create_model('convnext_base', pretrained=False, num_classes=10)
model.load_state_dict(torch.load(model_path))
model.to(device)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate=none)
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), e

In [14]:
# データの読み込み
test = pd.read_csv('sample_submit.tsv', header=None, sep='\t')
print(test.head())
print('データ数: ', test.shape[0])

            0  1
0  test_0.jpg  3
1  test_1.jpg  8
2  test_2.jpg  3
3  test_3.jpg  3
4  test_4.jpg  2
データ数:  8000


In [15]:
# 画像データの名前リストの抽出
x_test = test[0].values
dummy = test[0].values
print(x_test, f'データ数：{len(x_test)}')

['test_0.jpg' 'test_1.jpg' 'test_2.jpg' ... 'test_7997.jpg'
 'test_7998.jpg' 'test_7999.jpg'] データ数：8000


In [17]:
# transformの設定
transform = {
    'train': transforms.Compose([
        transforms.Resize(96),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomRotation(degrees=20),
        transforms.RandomAffine(degrees=[-10, 10], translate=(0.1, 0.1), scale=(0.5, 1.5)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    'val': transforms.Compose([
        transforms.Resize(96),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
}

In [18]:
# Datasetの設定
class CifarDataset(Dataset):
    def __init__(self, image_name_list, label_list, img_dir, transform=None, phase=None):
        self.image_name_list = image_name_list # 画像ファイル名
        self.label_list = label_list # ラベル
        self.img_dir = img_dir # 画像データのディレクトリ
        self.phase = phase # 変数phaseで学習(train)もしくは検証(val)の設定を行う
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list) # 1エポックあたりに読み込むデータ数として、入力データの数を指定

    def __getitem__(self, index):
        
        # index番目の画像を読み込み、前処理を行う
        image_path = os.path.join(self.img_dir, self.image_name_list[index]) # train_master.iloc[index, 0]はファイル名を抽出
        img = Image.open(image_path)
        img = self.transform[self.phase](img)
        
        # index番目のラベルを取得する
        label = self.label_list[index]
        
        return img, label

In [20]:
# Datasetのインスタンス作成
test_dataset = CifarDataset(x_test, dummy, 'test_images/', transform=transform, phase='val')

In [21]:
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [22]:
# 予測データフレームの作成
preds = []

# dataloaderから、ミニバッチ単位でデータを読み込む
for images, _ in test_dataloader:
    
    # 入力データをdeviceへ
    images = images.to(device)
    
    # 学習済みモデルを推論モードに設定
    model.eval()
    
    # モデルによる変換
    outputs = model(images)
    pred = torch.argmax(outputs, dim=1)
    pred = pred.to('cpu').numpy()

    # 予測値をリストに追加
    preds.extend(pred)


In [23]:
# 提出用データの読み込み
sub = pd.read_csv('sample_submit.tsv', header=None, sep='\t')
print(sub.head())
print('データ数: ', sub.shape[0])

            0  1
0  test_0.jpg  3
1  test_1.jpg  8
2  test_2.jpg  3
3  test_3.jpg  3
4  test_4.jpg  2
データ数:  8000


In [24]:
# 目的変数カラムの置き換え
sub[1] = preds
print(sub.head())

            0  1
0  test_0.jpg  5
1  test_1.jpg  7
2  test_2.jpg  5
3  test_3.jpg  8
4  test_4.jpg  3


In [26]:
# ファイルのエクスポート
sub.to_csv('/content/drive/MyDrive/Colab Notebooks/cifar10/submit/sub_test.tsv', sep='\t', header=None, index=None)